In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os

path = "./data"
file = 'test'

data = pd.read_csv(os.path.join(path, f'{file}.csv'), 
                   parse_dates=['일시'])
buildings = pd.read_csv(os.path.join(path, 'building_info.csv'))

In [3]:
print(data.isnull().sum())
print(buildings.isnull().sum())

num_date_time    0
건물번호             0
일시               0
기온(C)            0
강수량(mm)          0
풍속(m/s)          0
습도(%)            0
dtype: int64
건물번호            0
건물유형            0
연면적(m2)         0
냉방면적(m2)        0
태양광용량(kW)       0
ESS저장용량(kWh)    0
PCS용량(kW)       0
dtype: int64


In [4]:
data = pd.merge(data, buildings, on=['건물번호'], how='left')

In [5]:
if file == 'train':
    data.columns = ['building_id', 'num_date_time', 'Time', 'Temperature', 'Precipitation', 'Wind speed', 'Humidity', 'Sunshine', 'Solar radiation', 'consumption', 'type', 'Total area', 'Cooling area', 'Solar', 'ESS', 'PCS']
else:
    data.columns = ['num_date_time', 'building_id', 'Time', 'Temperature', 'Precipitation', 'Wind speed', 'Humidity', 'type', 'Total area', 'Cooling area', 'Solar', 'ESS', 'PCS']

In [6]:
for s, n in zip(data.type.unique(), range(12)):
    data.loc[data['type'] == s, 'type'] = n

In [7]:
data.loc[data.Solar == '-','Solar'] = 0
data.loc[data.ESS == '-', 'ESS'] = 0
data.loc[data.PCS == '-', 'PCS'] = 0

data['Wind speed'] = data['Wind speed'].interpolate(method='linear')
data['Humidity'] = data['Humidity'].interpolate(method='linear')
data['DI'] = 1.8 * data['Temperature'] - 0.55 * (1 - (data['Humidity']/100))*(1.8 * data['Temperature']-26) + 32

data.loc[data['Precipitation'] != data['Precipitation'], 'Precipitation'] = 0
data = data.fillna(-0.1)

print(data.isnull().sum())

num_date_time    0
building_id      0
Time             0
Temperature      0
Precipitation    0
Wind speed       0
Humidity         0
type             0
Total area       0
Cooling area     0
Solar            0
ESS              0
PCS              0
DI               0
dtype: int64


In [8]:
data = data.drop(['num_date_time', ], axis=1)
# data = data.drop('type', axis=1)

date = pd.to_datetime(data.Time)

data['hour'] = data['Time'].dt.hour
data['day'] = data['Time'].dt.weekday
data['date'] = data['Time'].dt.date.astype('str')
data['month'] = data['Time'].dt.month

In [9]:
data['holiday'] = np.zeros(len(data))

holidays = ['2022-06-06', '2022-07-17', '2022-08-15']
data['holiday'] = np.zeros(len(data)).astype(int)
data.loc[data.day >= 5, 'holiday'] = 1
for holiday in holidays:
    data.loc[data.date == holiday, 'holiday'] = 1

In [10]:
# clusters = {0: [4,10,16,17,18,19,20,21,22,23,24,25,26,27,28,30,44,45,46,47,48,49,50,51,52,54,55,57,58,59,68,69,71,73,74,75,76,77,78,79,81,82,83],
#             1: [0, 1,3,5,6,8,11,15,36,37,38,39,40,41,42,43,53,56,80,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99],
#             2: [2, 60, 61, 62, 63, 64, 65, 66, 67],
#             3: [7, 9, 12, 13, 14, 29, 31, 32, 33, 34, 35, 70, 72]}

# for cluster in clusters:
#     ids = [c + 1 for c in clusters[cluster]]
#     data.loc[data.building_id.isin(ids), 'cluster'] = cluster

In [11]:
if file == 'train':
    #######################################
    ## 건물별, 요일별, 시간별 발전량 평균 넣어주기
    #######################################

    power_mean = pd.pivot_table(data, values = 'consumption', index = ['building_id', 'hour', 'day'], aggfunc = np.mean).reset_index()
    data['day_hour_mean'] = data.apply(lambda x : power_mean.loc[(power_mean.building_id == x['building_id']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'consumption'].values[0], axis = 1)

    #######################################
    ## 건물별 시간별 발전량 평균 넣어주기
    #######################################
    power_hour_mean = pd.pivot_table(data, values = 'consumption', index = ['building_id', 'hour'], aggfunc = np.mean).reset_index()
    data['hour_mean'] = data.apply(lambda x : power_hour_mean.loc[(power_hour_mean.building_id == x['building_id']) & (power_hour_mean.hour == x['hour']) ,'consumption'].values[0], axis = 1)

    #######################################
    ## 건물별 시간별 발전량 표준편차 넣어주기
    #######################################
    power_hour_std = pd.pivot_table(data, values = 'consumption', index = ['building_id', 'hour'], aggfunc = np.std).reset_index()
    data['hour_std'] = data.apply(lambda x : power_hour_std.loc[(power_hour_std.building_id == x['building_id']) & (power_hour_std.hour == x['hour']) ,'consumption'].values[0], axis = 1)

## https://dacon.io/competitions/official/235680/codeshare/2366?page=1&dtype=recent
data['sin_time'] = np.sin(2*np.pi*data.hour/24)
data['cos_time'] = np.cos(2*np.pi*data.hour/24)

def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

cdhs = np.array([])
for num in range(1,101,1):
    temp = data[data['building_id'] == num]
    cdh = CDH(temp['Temperature'].values)
    cdhs = np.concatenate([cdhs, cdh])
data['CDH'] = cdhs

data.drop(['date', 'type', 'Total area', 'Cooling area','Solar','hour', 'PCS', 'ESS', 'Sunshine'], axis = 1, inplace = True)
data.head()

KeyError: "['Sunshine'] not found in axis"

In [12]:
if file == 'train':
       data = data[['building_id', 'Time', 'sin_time', 'cos_time', 'day', 'month', 'day_hour_mean', 'hour_mean', 'hour_std', 'holiday', 'Temperature', 'Precipitation', 'Wind speed', 'Humidity', 'DI', 'Solar radiation', 'consumption', 'CDH']]
else:
       data = data[['building_id', 'Time', 'sin_time', 'cos_time', 'day', 'month', 'holiday', 'Temperature', 'Precipitation', 'Wind speed', 'Humidity', 'DI', 'CDH']]

In [13]:
data

,building_id,Time,sin_time,cos_time,day,month,holiday,Temperature,Precipitation,Wind speed,Humidity,DI,CDH
0,1,2022-08-25 00:00:00,0.000000,1.000000,3,8,0,23.5,0.0,2.2,72,71.78980,-2.5
1,1,2022-08-25 01:00:00,0.258819,0.965926,3,8,0,23.0,0.0,0.9,72,71.02840,-5.5
2,1,2022-08-25 02:00:00,0.500000,0.866025,3,8,0,22.7,0.0,1.5,75,70.81675,-8.8
3,1,2022-08-25 03:00:00,0.707107,0.707107,3,8,0,22.1,0.0,1.3,78,70.11262,-12.7
4,1,2022-08-25 04:00:00,0.866025,0.500000,3,8,0,21.8,0.0,1.0,77,69.56514,-16.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,100,2022-08-31 19:00:00,-0.965926,0.258819,2,8,0,22.5,0.0,0.9,84,71.22400,-34.5
16796,100,2022-08-31 20:00:00,-0.866025,0.500000,2,8,0,20.7,0.0,0.4,95,68.95035,-34.4
16797,100,2022-08-31 21:00:00,-0.707107,0.707107,2,8,0,20.2,0.0,0.4,98,68.24604,-35.3
16798,100,2022-08-31 22:00:00,-0.500000,0.866025,2,8,0,20.1,0.0,1.1,97,68.01203,-36.8


In [14]:
data.to_csv(os.path.join(path, f'preproc3_{file}.csv'), index=False)

In [15]:
pd.read_csv(os.path.join(path, f'preproc3_{file}.csv')).head()

,building_id,Time,sin_time,cos_time,day,month,holiday,Temperature,Precipitation,Wind speed,Humidity,DI,CDH
0,1,2022-08-25 00:00:00,0.000000,1.000000,3,8,0,23.5,0.0,2.2,72,71.78980,-2.5
1,1,2022-08-25 01:00:00,0.258819,0.965926,3,8,0,23.0,0.0,0.9,72,71.02840,-5.5
2,1,2022-08-25 02:00:00,0.500000,0.866025,3,8,0,22.7,0.0,1.5,75,70.81675,-8.8
3,1,2022-08-25 03:00:00,0.707107,0.707107,3,8,0,22.1,0.0,1.3,78,70.11262,-12.7
4,1,2022-08-25 04:00:00,0.866025,0.500000,3,8,0,21.8,0.0,1.0,77,69.56514,-16.9
